In [ ]:
# riboswitch and ribosnitch strategy


In [ ]:
## 1. Figure_2b ---------------------

In [28]:
import os 
import glob
import pandas as pd 
import numpy as np
from scipy import io as sio 
from utils_io import filter_data_by_modrate 
from dreem import DREEM 

In [ ]:
### 1_1. get the input reads ---

In [29]:
def get_sizes(f_sizes):
    sizes = {}
    with open(f_sizes, "r") as f:
        for line in f:
            row = line.strip("\r\n").split("\t")
            sizes[row[0]] = int(row[1])
    return sizes 

def convert_mm_to_df(f_mtx, f_id, p_gene, 
                     f_sizes, p_sep=",", p_threshold_len_prop=0.8):
    """
    Utility function for processing sparse matrix .mtx files
    """

    # load sizes file
    sizes = get_sizes(f_sizes)
    
    # load iid file
    if pd.read_csv(f_id, index_col=0, sep=p_sep).shape[1] == 1:
        df_iid = pd.read_csv(f_id, index_col=0, sep=p_sep, names=["iid"])
    else:
        df_iid = pd.read_csv(f_id, index_col=0, sep=p_sep, names=["iid", "length", "start"])
    
    # load sparse matrix
    sm = sio.mmread(f_mtx)

    # determine whether sparse matrix is undersized and remedy if so
    new_row_size = df_iid.shape[0]
    new_column_size = sizes[p_gene]
    if sm.shape[1] < new_column_size:  # resize
        sm.resize((new_row_size, new_column_size))
    elif sm.shape[1] > new_column_size:
        print(sm.shape[1], new_column_size)
        print("shapemap column greater than genome sizes file")
        assert False

    # load sparse matrix as a pandas dataframe
    df_mm = pd.DataFrame.sparse.from_spmatrix(sm)
    df_mm.index = df_iid.iid

    if df_iid.shape[1] >= 2:
        # Threshold by Length
        df_mm = df_mm.loc[df_iid[df_iid.length >= (p_threshold_len_prop * sizes[p_gene])].iid,:]
        #df_mm = df_mm.loc[df_iid[df_iid.length >= 300].iid,:]
    
    return df_mm


def load_data(f_mtxs, f_ids, p_genes, f_sizes=None, p_depth=-1, p_length=-1, 
              p_start=None, p_end=None, p_threshold=1,
              p_threshold_len_prop=0.8, p_verbose=True):
    
    """
    Loads SHAPE-MaP or PORE-cupine data based on the input specifications
    Must be able to accept both dense matrix or sparse matrix
    """

    dfs = []
    for f_mtx, f_id, p_gene in zip(f_mtxs, f_ids, p_genes):
        df = convert_mm_to_df(f_mtx, f_id, p_gene,
                            f_sizes=f_sizes,
                            p_threshold_len_prop=p_threshold_len_prop)
        dfs.append(df)
    
    
    if p_length == -1:
        p_start = 0
        p_end = dfs[0].shape[1]

    if p_depth == -1:
        p_depth = min([ df.shape[0] for df in dfs ])

    if p_verbose:
        print("%s %s %s" % (p_start, p_end, " ".join(map(str, [ df.shape[0] for df in dfs]))))
        print(p_depth)

    if p_depth is None:
        ndfs =  [ df.iloc[:,p_start:p_end] for df in dfs ]
    else:
        ndfs = [ df.iloc[:p_depth,p_start:p_end] for df in dfs ]
    
    ndf = pd.concat(ndfs, axis=0)

    X = ndf
    X = X.loc[X.sum(axis=1)>=p_threshold,:]
    X = X.fillna(0)

    return X, ndfs


def convert_to_dreem(sid,seq,p_length,f_output):
        with open(f_output, "w") as f:
            f.write("\t".join(["@ref", "%s;%s" % (sid, sid),seq]) + "\n")
            f.write("\t".join(["@coordinates:length", "1,%s:%s" % (p_length,p_length)]) + "\n")
            f.write("\t".join(["Query_name", "Bit_vector", "N_Mutations"]) + "\n")
            for no, d in enumerate(library):
                no_of_mutations = d.sum()
                f.write("\t".join(["%s.1" % no,
                                "".join(map(str, d)),
                                str(no_of_mutations)]) + "\n")
        return True 

def write_to_fasta(sid,seq,f_output):
        with open(f_output, "w") as f:
            f.write(">%s\n" % sid)
            f.write("%s\n" % seq)
        return True 

In [30]:
p_genes = ["ADD-TRUNC"]
p_mod = "NAIN3"
p_depth = None
p_length = -1
p_start = None
p_end = None
p_threshold_len_prop = 0.85
p_verbose = True

f_sizes = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/reference_of_ribosxitch_v2.sizes"

### WT ----
f_dir = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/"
f_mtxs = [f_dir + "NOADD_DMS-MaP_DMS_B01_raw.mtx"]
f_ids =  [ f_dir + "NOADD_DMS-MaP_DMS_B01_raw.iids.gz"]


X_WT, df_WTs  = load_data(f_mtxs, f_ids, p_genes, f_sizes=f_sizes, 
                                  p_depth=p_depth, p_length=p_length, 
                                  p_start=p_start, p_end=p_end, 
                                  p_threshold_len_prop=p_threshold_len_prop, 
                                  p_verbose=p_verbose)

### MT ----
f_dir = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/"
f_mtxs = [f_dir + "ADD_DMS-MaP_DMS_B01_raw.mtx"]
f_ids =  [f_dir + "ADD_DMS-MaP_DMS_B01_raw.iids.gz"]

X_MT, df_MTs  = load_data(f_mtxs, f_ids, p_genes, f_sizes=f_sizes, 
                                  p_depth=p_depth, p_length=p_length, 
                                  p_start=p_start, p_end=p_end, 
                                  p_threshold_len_prop=p_threshold_len_prop, 
                                  p_verbose=p_verbose)

df_WT = pd.concat(df_WTs)
df_MT = pd.concat(df_MTs)
X = pd.concat([X_WT, X_MT])

p_threshold_len_prop=0.85
p_modrate_low_pos=0.0
p_modrate_high_pos=1.0
p_modrate_low_read=0.0075
p_modrate_high_read=0.025
p_min_samples = 1000
p_threshold = 0.5

# Filter reads and positions by min/max modification rate
df_WT_flt = filter_data_by_modrate(df_WT, 
                                    p_modrate_low=p_modrate_low_pos, 
                                    p_modrate_high=p_modrate_high_pos, p_axis=0)  # this will zero the data instead of remove
df_MT_flt = filter_data_by_modrate(df_MT, 
                                    p_modrate_low=p_modrate_low_pos, 
                                    p_modrate_high=p_modrate_high_pos, p_axis=0)  # this will zero the data instead of remove

df_WT_flt = filter_data_by_modrate(df_WT_flt, p_modrate_low=p_modrate_low_read, p_modrate_high=p_modrate_high_read, p_axis=1)
df_MT_flt = filter_data_by_modrate(df_MT_flt, p_modrate_low=p_modrate_low_read, p_modrate_high=p_modrate_high_read, p_axis=1)

print(df_WT_flt.shape, df_MT_flt.shape)


p_min_samples = 1000

# BMM
np.random.seed(386) #386, 472, 123, 823
    
if p_min_samples == -1:
    min_samples = min(df_WT_flt.shape[0], df_MT_flt.shape[0]) #2900
else:
    min_samples = p_min_samples
    
total_reads = p_min_samples*2
prop=0.5
WT_reads = total_reads*prop
MT_reads = total_reads-WT_reads
df_WT_flt2 = df_WT_flt.sample(int(WT_reads), random_state=386)
df_MT_flt2 = df_MT_flt.sample(int(MT_reads), random_state=386)

new_X = pd.concat([df_WT_flt2, df_MT_flt2])

df_WT_flt2['group'] = 'Noligand'
df_MT_flt2['group'] = 'Ligand'
new_X2 = pd.concat([df_WT_flt2, df_MT_flt2])
print(new_X.shape)


truths = [0]*1000 + [1]*1000
data =np.array(new_X)
sid = 'ADD'
seq = 'GGACACGACTCGAGTAGAGTCGAATGCGCTTCATATAATCCTAATGATATGGTTTGGGAGTTTCTACCAAGAGCCTTAAACTCTTGATTATGAAGTCTGTCGCTTTATCCGAAATTTTATAAAGAGAAGACTCATGAATTACTTTGACCTGCCGACCGGAGTCGAGTAGACTCCAACAAAAGAAACAACAACAACAAC'
p_length = 198
library= data 
print(len(seq)) 
dreem_data=convert_to_dreem(sid,seq,p_length,"/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/result/input_dreem.txt")
write_to_fasta(sid,seq,"/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/result/input_fasta.fasta")

0 198 148814
None
0 198 349789
None


/shared/users/hanjian/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/utils_io.py:229: FutureWarning:

In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`



(111750, 198) (229784, 198)
(2000, 198)
198


True

In [ ]:
### 1_2. run DREEM model ---

In [31]:
f_dreem_prefix = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/result/"
dir_input = f_dreem_prefix + "input/"
os.system("mkdir -p %s" % dir_input)
dir_output = "./"
dir_outplot = f_dreem_prefix + ("output/%s/" % sid)
os.system("mkdir -p %s" % dir_outplot)

p_cluster=2
p_seed = 386
model = DREEM(sid, 
              data, 
              dir_input, dir_output, dir_outplot,
              MIN_ITS=3, INFO_THRESH=0.05, CONV_CUTOFF=0.5, NUM_RUNS=1,  # 1e-5
              MAX_K=p_cluster, SIG_THRESH=0.001, BV_THRESH=0, 
              NORM_PERC_BASES=10, inc_TG=True, p_seed=p_seed)

# Fit model and Parse DREEM results into DREEM object
model.fit()

Mutations threshold: 20
Total bit vectors: 2000
Bit vectors removed because of too many mutations:  0
Bit vectors removed because of too few informative bits:  0
Bit vectors removed because of mutations close by:  0
Bit vectors removed because of no info around mutations:  0
Working on K = 2
Run number: 1
3 -23571.016549024385 46.429388306143665 -0.001969766056104415
4 -23533.16860468245 37.847944341934635 -0.0016082808472465515
5 -23490.59268111755 42.57592356490204 -0.0018124669795635195
6 -23444.56002038139 46.032660736160324 -0.0019634687405582405
7 -23402.449262439688 42.110757941700285 -0.001799416696494539
8 -23370.106458266728 32.342804172960314 -0.001383939103175102
9 -23348.127165631406 21.979292635322054 -0.0009413728338637676
10 -23334.598164635063 13.529000996342802 -0.0005797829000906812
11 -23326.61323248457 7.984932150491659 -0.0003423099646275211
12 -23322.00418043111 4.609052053459891 -0.00019762675702319043
13 -23319.286609908973 2.717570522137976 -0.0001165374639283

/apps/users/hanjian/miniconda3/envs/gis/lib/python3.9/site-packages/plotly/tools.py:461: DeprecationWarning:

plotly.tools.make_subplots is deprecated, please use plotly.subplots.make_subplots instead



In [ ]:
### 1_3. process DREEM result ---

In [32]:
def parse_dreem_results(f_result, p_cluster, library, truths, p_threshold=0.5):
        datum_pred = []
        dict_pred = {}
        with open(f_result, "r") as f:
            for no, line in enumerate(f):
                if no % 2 == 1:
                    row = line.strip("\r\n").split("\t")
                    rid, posteriors, bitvector = row[0], list(map(float, row[1:1+p_cluster])), row[-1]
                    cluster_no = np.argmax(posteriors)  # max posterior
                    datum_pred.append(cluster_no)
                    dict_pred[bitvector] = cluster_no

        new_datum_truth = []
        new_datum_pred = []
        new_data = []
        new_index = []
        for no, (d, t) in enumerate(zip(library, truths)):
            try:
                p = dict_pred["".join(map(str, d.tolist()))]
                new_datum_truth.append(t)
                new_datum_pred.append(p)
                new_data.append(d)
                new_index.append(no)
            except KeyError:
                pass
        new_datum_truth = np.array(new_datum_truth)
        new_datum_pred = np.array(new_datum_pred)
        new_data = np.array(new_data)
        new_index = np.array(new_index)

        return new_datum_pred, new_datum_truth, new_data, new_index, dict_pred

In [33]:
truths = [0]*1000 + [1]*1000
new_datum_pred, new_datum_truth, new_data, new_index,dict_pred = parse_dreem_results("/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Figure2/Data/DMS-MaP/result/output/ADD/K_2/run_1-best/Responsibilities.txt", 
                                                                           2, 
                                                                           library,
                                                                           truths,
                                                                           p_threshold=0.5)

df = pd.DataFrame(list(zip(list(new_datum_pred), list(new_datum_truth))), columns=['pred_cluster','truth_cluster'])
df.groupby(['pred_cluster'])['truth_cluster'].value_counts()

pred_cluster  truth_cluster
0             1                699
              0                223
1             0                774
              1                301
Name: truth_cluster, dtype: int64